In [37]:
#Import libraries for data extraction
import csv
import requests 
from bs4 import BeautifulSoup
import pandas as pd
from re import search

In [22]:
#Enter url for desired site
url = "https://racedaytiming.ca/results/2017USPORTSXCMen"

In [23]:
#Retrieve and process site information
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [24]:
#Preview site as HTML content
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-ca">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="" name="description"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Race Day Timing
  </title>
  <!-- Disable tap highlight on IE -->
  <meta content="no" name="msapplication-tap-highlight"/>
  <!-- Add to homescreen for Chrome on Android -->
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="Web Starter Kit" name="application-name"/>
  <link href="/images/touch/chrome-touch-icon-192x192-2.png" rel="icon" sizes="192x192"/>
  <!-- Add to homescreen for Safari on iOS -->
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <meta content="black" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="Web Starter Kit" name="apple-mobile-web-app-title"/>
  <link href="/images/touch/apple-touch-icon-2.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/images/favico

In [25]:
#Get all instances of tage that would contain relevant data
soup.find_all('a')

[<a class="navbar-brand mr-auto" href="/">
 <img alt="RaceDay Timing" class="main-logo img-fluid" height="50px" src="/images/racedaytiming-logo-white.svg" width="227px"/>
 </a>,
 <a class="nav-link" href="/results.html">Results</a>,
 <a class="nav-link" href="/series-results.html">Series Results</a>,
 <a class="nav-link" href="/event-services.html">Event Services</a>,
 <a class="nav-link" href="/participant-services.html">Participant Services</a>,
 <a class="nav-link" href="/contact.html">Contact</a>,
 <a class="btn btn-primary login-btn" href="/account">Log In </a>,
 <a class="active" href="/results/2017USPORTSXCMen">FINAL</a>,
 <a class="" href="/results/2017-11-09-15-50-20">LAP 4</a>,
 <a class="" href="/results/2017USPORTSXCMenLap3">LAP 3</a>,
 <a class="" href="/results/2017USPORTSXCMenLap2">LAP 2</a>,
 <a class="" href="/results/2017USPORTSXCMenLap1">LAP 1</a>,
 <a class="race-info-website" href="https://usports.ca/en/championships/cross-country/x">Event Website</a>,
 <a class="r

We can see that from a bit of scrolling we find the elements that contain the athlete's name and rank. <br> This gives us the class as well which is "full-results-btn", which we can use to filter down our search.

Note: This is specific to this site and will only work for records retrieved from this site.

In [26]:
#Get all instances filtered by class
soup.find_all('a', class_='full-results-btn')

[<a class="full-results-btn" href="/results/2017USPORTSXCMen/Yves+Sikubwabo_1"><span>View full <br/> results</span><i aria-hidden="true" class="fa fa-chevron-right"></i></a>,
 <a class="full-results-btn" href="/results/2017USPORTSXCMen/Connor+Black_2"><span>View full <br/> results</span><i aria-hidden="true" class="fa fa-chevron-right"></i></a>,
 <a class="full-results-btn" href="/results/2017USPORTSXCMen/Ben+Carson_3"><span>View full <br/> results</span><i aria-hidden="true" class="fa fa-chevron-right"></i></a>,
 <a class="full-results-btn" href="/results/2017USPORTSXCMen/Russell+Pennock_4"><span>View full <br/> results</span><i aria-hidden="true" class="fa fa-chevron-right"></i></a>,
 <a class="full-results-btn" href="/results/2017USPORTSXCMen/Alexander+James_5"><span>View full <br/> results</span><i aria-hidden="true" class="fa fa-chevron-right"></i></a>,
 <a class="full-results-btn" href="/results/2017USPORTSXCMen/Dany+Racine_6"><span>View full <br/> results</span><i aria-hidden="t

In [27]:
#Get required text from these instances
links_with_text = []
for a in soup.find_all('a', class_='full-results-btn', href=True): 
    if a.text: 
        links_with_text.append(a['href'])

In [28]:
#Preview first twenty text values obtained
links_with_text[:20]

['/results/2017USPORTSXCMen/Yves+Sikubwabo_1',
 '/results/2017USPORTSXCMen/Connor+Black_2',
 '/results/2017USPORTSXCMen/Ben+Carson_3',
 '/results/2017USPORTSXCMen/Russell+Pennock_4',
 '/results/2017USPORTSXCMen/Alexander+James_5',
 '/results/2017USPORTSXCMen/Dany+Racine_6',
 '/results/2017USPORTSXCMen/Eric+Wynands_7',
 '/results/2017USPORTSXCMen/Sergio+Raez+Villanueva_8',
 '/results/2017USPORTSXCMen/Mark+Patton_9',
 '/results/2017USPORTSXCMen/Angus+Rawling_10',
 '/results/2017USPORTSXCMen/Shawn+Master_11',
 '/results/2017USPORTSXCMen/Declan+White_12',
 '/results/2017USPORTSXCMen/Andrew+Nebel_13',
 '/results/2017USPORTSXCMen/Stefan+Daniel_14',
 '/results/2017USPORTSXCMen/Andrew+Shepherd_15',
 '/results/2017USPORTSXCMen/Benjamin+Workman_16',
 '/results/2017USPORTSXCMen/Jean-Simon+Desgagnes_17',
 '/results/2017USPORTSXCMen/Braydon+Clarke_18',
 '/results/2017USPORTSXCMen/Joshua+McGillivray_19',
 '/results/2017USPORTSXCMen/Max+Turek_20']

In [29]:
#Now, we will do this for all the availible pages
links_with_text = []
for pg in range(1,4):
    url = "https://racedaytiming.ca/results/2017USPORTSXCMen?page={}".format(pg)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    for a in soup.find_all('a', class_='full-results-btn', href=True): 
        if a.text: 
            links_with_text.append(a['href'])

In [30]:
#Preview completed list of text data from pages
links_with_text

['/results/2017USPORTSXCMen/Yves+Sikubwabo_1',
 '/results/2017USPORTSXCMen/Connor+Black_2',
 '/results/2017USPORTSXCMen/Ben+Carson_3',
 '/results/2017USPORTSXCMen/Russell+Pennock_4',
 '/results/2017USPORTSXCMen/Alexander+James_5',
 '/results/2017USPORTSXCMen/Dany+Racine_6',
 '/results/2017USPORTSXCMen/Eric+Wynands_7',
 '/results/2017USPORTSXCMen/Sergio+Raez+Villanueva_8',
 '/results/2017USPORTSXCMen/Mark+Patton_9',
 '/results/2017USPORTSXCMen/Angus+Rawling_10',
 '/results/2017USPORTSXCMen/Shawn+Master_11',
 '/results/2017USPORTSXCMen/Declan+White_12',
 '/results/2017USPORTSXCMen/Andrew+Nebel_13',
 '/results/2017USPORTSXCMen/Stefan+Daniel_14',
 '/results/2017USPORTSXCMen/Andrew+Shepherd_15',
 '/results/2017USPORTSXCMen/Benjamin+Workman_16',
 '/results/2017USPORTSXCMen/Jean-Simon+Desgagnes_17',
 '/results/2017USPORTSXCMen/Braydon+Clarke_18',
 '/results/2017USPORTSXCMen/Joshua+McGillivray_19',
 '/results/2017USPORTSXCMen/Max+Turek_20',
 '/results/2017USPORTSXCMen/Brandon+Vail_21',
 '/resu

In [31]:
#Get general text string info
txt = links_with_text[0]

#Length of string
print(txt.split('/'))
print(txt.split('/')[0])
print(txt.split('/')[1])
print(txt.split('/')[2])
print(txt.split('/')[3])

['', 'results', '2017USPORTSXCMen', 'Yves+Sikubwabo_1']

results
2017USPORTSXCMen
Yves+Sikubwabo_1


In [32]:
#Parse string values to get valid text data
year_event = []
name_rank = []
for text in links_with_text:
    nr = text.split("/")[3]
    ye = text.split("/")[2]
    name_rank.append(nr)
    year_event.append(ye)

In [33]:
#Add information to lists
year = []
event = []
for text in year_event:
    y = text[0:4]
    e = text[4:]
    year.append(y)
    event.append(e)

In [44]:
#Separate and assign data from string
f_name = []
l_name = []
rank = []
for text in name_rank:
    fn = text.split("+")[0]
    ln_r = text.split("+")[-1]
    ln = ln_r.split("_")[0]
    r = ln_r.split("_")[-1]
    f_name.append(fn)
    l_name.append(ln)
    rank.append(r)

In [45]:
#Clean up string entries
for name in l_name:
    apost = "%27"
    if search(apost, name):
        name = name.replace(apost, "'")

In [46]:
#Construct dataframe using entry lists 
dict = {'Year': year, 'Event': event, 'First Name': f_name, 'Last Name': l_name, 'Rank': rank}  
df = pd.DataFrame(dict) 

In [49]:
#Preview Dataframe
df.tail(10)

,Year,Event,First Name,Last Name,Rank
266,2017,USPORTSXCMen,Louis-Olivier,Brassard,129
267,2017,USPORTSXCMen,Connor,Boyd,130
268,2017,USPORTSXCMen,Rhys,Desmond,131
269,2017,USPORTSXCMen,Alexander,McBride,132
270,2017,USPORTSXCMen,Bryan,Thomas,133
271,2017,USPORTSXCMen,Conlan,Sprickerhoff,134
272,2017,USPORTSXCMen,Jordan,Gin,135
273,2017,USPORTSXCMen,Jean-Michel,Robichaud,136
274,2017,USPORTSXCMen,Chris,Moore,137
275,2017,USPORTSXCMen,Nik,Rampen,138


In [57]:
#Export data in csv
filename = str(year[0]) + " " + str(event[0]) + " "  + "Data"
file = df.to_csv(filename,index=False)